<a href="https://colab.research.google.com/github/kvnchandra/ml-assignment-bangkit/blob/citra/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicting Breast Cancer Diagnostic**


---


*Bangkit Academy 2020 : DPS-1 #05-W05-First Machine Learning Project - MLCC*

Team members :
1.   Kevin Chandra
2.   Oktavia Citra
3.   Gani Nur Fadli

We use data from [kaggle.com ](https://https://www.kaggle.com/) with the title "Breast Cancer Wisconsin (Diagnostic) Data Set". We want to predict whether the cancer is benign or malignant of breast cancer patients. We can build this program because we've learned a lot from [Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course) and [TensorFlow Website](https://www.tensorflow.org/tutorials/)

***Here the step by step and code bellow!***


# Setup

identification tensorflow newest version, so there is no error with the caption "tensorflow is already loaded. please restart the runtime to change versions."

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


import all modules and library that you need in the top

In [0]:
# Import relevant modules
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import numpy as np
import pandas as pd
import tensorflow as tf